In [ ]:
# 1. single approach that predicts N edges using max_balance in one run
# and then predict using lowrank method

# 2. iterative approach that predicts N edges in X runs
# and then predict using LR method


# result:
# plot of two lines: 
# x-axis: N, minimum triangle count threshold (similar to embeddedness)
# y-axis: accuracy

In [1]:

import _pickle as pkl
import networkx as nx
import numpy as np
import random

from tqdm import tqdm
from snpp.cores.lowrank import alq_spark, predict_signs
from snpp.utils.matrix import split_train_test, load_sparse_csr
from snpp.utils.signed_graph import g2m
from snpp.utils.data import load_train_test_graphs
from snpp.utils.edge_filter import filter_by_min_triangle_count

from snpp.utils.spark import sc

dataset = 'slashdot'
lambda_ = 0.2
k = 10
max_iter = 100
random_seed = 123456
min_tri_count = 20

recache_input = False

random.seed(random_seed)
np.random.seed(random_seed)

In [2]:
train_g, test_g = load_train_test_graphs(dataset, recache_input)
train_g_ud = train_g.to_undirected()

loading train and test graphs...


In [3]:
confident_edges = set(filter_by_min_triangle_count(train_g_ud, test_g.edges(), min_tri_count))

In [4]:
from snpp.cores.joint_part_pred import iterative_approach
from snpp.cores.max_balance import faster_greedy
from snpp.cores.lowrank import partition_graph
from snpp.cores.budget_allocation import constant_budget
from snpp.cores.triangle import build_edge2edges

common_params = dict(
    g=train_g_ud,
    T=confident_edges,
    k=k,
    graph_partition_f=partition_graph,
    graph_partition_kwargs=dict(sc=sc,
                                lambda_=lambda_,
                                iterations=max_iter,
                                seed=random_seed),
    budget_allocation_f=constant_budget,
    solve_maxbalance_f=faster_greedy,
    solve_maxbalance_kwargs={'edge2edges': build_edge2edges(train_g_ud.copy(),
                                                            confident_edges)},
    truth=set([(i, j, test_g[i][j]['sign'])
               for i, j in confident_edges]),
    perform_last_partition=False
)

  6%|▋         | 171/2645 [00:00<00:01, 1709.96it/s]

build edge2edges


100%|██████████| 2645/2645 [00:01<00:00, 2020.62it/s]


In [5]:
from snpp.utils.evaluation import accuracy

In [ ]:
# single iteration approach

part, single_preds, status = iterative_approach(
    budget_allocation_kwargs=dict(const=len(confident_edges)),
    **common_params
)
print(" => accuracy {} (single)".format(accuracy(test_g, single_preds)))


In [ ]:
# iterative approach

part, iter_preds, status = iterative_approach(
    budget_allocation_kwargs=dict(const=200),
    **common_params
)
print(" => accuracy {} (iterative)".format(accuracy(test_g, iter_preds)))


In [ ]:
# partition and cut approach

from snpp.cores.joint_part_pred import single_run_approach

_, part_and_cut_preds = single_run_approach(train_g, 
                                            confident_edges,
                                            k,
                                            graph_partition_f=partition_graph,
                                            graph_partition_kwargs=dict(sc=sc,
                                                                    lambda_=lambda_,
                                                                    iterations=max_iter,
                                                                    seed=random_seed))
print(" => accuracy {} (partition-and-cut)".format(accuracy(test_g, part_and_cut_preds)))


to_scipy_sparse_matrix


100%|██████████| 77357/77357 [00:02<00:00, 29265.47it/s]


ALS...
